# Churn Prediction - Feature Engineering
---
**Table of Contents** :
1. Import packages and Define functions
2. Load data
3. Feature Engineering
    - Create variables that capture price differentials, reflecting customer price sensitivity
    - Transform `datetime` features into numeric values representing the number of months (as integers)
4. Data Preprocessing
   - Encoding Categorical Data
   - Addressing Skewness in Numerical Data
   - Correlation Analysis
5. Export the data for modeling

---
## 1. Import packages and Define functions

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math
from datetime import datetime
from scipy.stats import skew
# Shows plots in jupyter notebook
%matplotlib inline


# Set plot style
sns.set(color_codes=True)
plt.style.use('ggplot')

#### visualization function

In [3]:
def plot_stacked_bars(dataframe, title_, size_=(18, 10), rot_=0, legend_="upper right"):
    """
    Plot stacked bars with annotations
    """
    ax = dataframe.plot(
        kind="bar",
        stacked=True,
        figsize=size_,
        rot=rot_,
        title=title_
    )

    # Annotate bars
    annotate_stacked_bars(ax, textsize=14)
    # Rename legend
    plt.legend(dataframe.columns, loc=legend_)
    # Labels
    plt.ylabel("Company base (%)")
    plt.show()

def annotate_stacked_bars(ax, bins_, pad=0.99, colour="black", textsize=13):
    """
    Add value annotations to the bars
    """
    #print(len(ax.patches))
    # Iterate over the plotted rectanges/bars
    for i in range(len(ax.patches)):
        

        if (i<bins_) :
        # Calculate annotation
            bar_total_height = (ax.patches[i].get_height() + ax.patches[i+int(bins_)].get_height())
        else :
            bar_total_height = (ax.patches[i].get_height() + ax.patches[i-int(bins_)].get_height())

        
        value = (ax.patches[i].get_height()/ bar_total_height) * 100

        if (np.isnan(value) or value < 5 or (bar_total_height < 50)):
            continue
            
        value = str(round(value,1)) + '%'
        #print(value)
        
        ax.annotate(
            value,
            #((p.get_x()+ p.get_width()/2)*pad-0.05, (p.get_y()+p.get_height()/2)*pad),
            ((ax.patches[i].get_x()), (ax.patches[i].get_y()+ax.patches[i].get_height()/2)*pad),
            color=colour,
            size=textsize
        )

def plot_distribution(dataframe, column, ax, bins_=50):
    """
    Plot variable distirbution in a stacked histogram of churned or retained company
    """
    # Create a temporal dataframe with the data to be plot
    temp = pd.DataFrame({"Retention": dataframe[dataframe["churn"]==0][column],
    "Churn":dataframe[dataframe["churn"]==1][column]})

    #print(temp)
    # Plot the histogram
    temp[["Retention", "Churn"]].plot(kind='hist', bins=bins_, ax=ax, stacked=True)
    # X-axis label
    ax.set_xlabel(column)
    # Change the x-axis to plain style
    ax.ticklabel_format(style='plain', axis='x')
    annotate_stacked_bars(ax, bins_ = bins_, textsize=14)


---
## 2. Load data

In [4]:
client_df = pd.read_csv('./data/client_data.csv')
price_df = pd.read_csv('./data/price_data.csv')

In [5]:
client_df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,f,52.32,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0


In [6]:
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


### Dataset Description

**client_data** :
- id = client company identifier
- channel_sales = code of the sales channel
- cons_12m = electricity consumption of the past 12 months
- cons_gas_12m = gas consumption of the past 12 months
- cons_last_month = electricity consumption of the last month
- date_activ = date of activation of the contract
- date_end = registered date of the end of the contract
- date_modif_prod = date of the last modification of the product
- date_renewal = date of the next contract renewal
- forecast_cons_12m = forecasted electricity consumption for next 12 months
- forecast_cons_year = forecasted electricity consumption for the next calendar year
- forecast_discount_energy = forecasted value of current discount
- forecast_meter_rent_12m = forecasted bill of meter rental for the next 2 months
- forecast_price_energy_off_peak = forecasted energy price for 1st period (off peak)
- forecast_price_energy_peak = forecasted energy price for 2nd period (peak)
- forecast_price_pow_off_peak = forecasted power price for 1st period (off peak)
- has_gas = indicated if client is also a gas client
- imp_cons = current paid consumption
- margin_gross_pow_ele = gross margin on power subscription
- margin_net_pow_ele = net margin on power subscription
- nb_prod_act = number of active products and services
- net_margin = total net margin
- num_years_antig = antiquity of the client (in number of years)
- origin_up = code of the electricity campaign the customer first subscribed to
- pow_max = subscribed power
- churn = has the client churned over the next 3 months


**price_data**:
- id = client company identifier
- price_date = reference date
- price_off_peak_var = price of energy for the 1st period (off peak)
- price_peak_var = price of energy for the 2nd period (peak)
- price_mid_peak_var = price of energy for the 3rd period (mid peak)
- price_off_peak_fix = price of power for the 1st period (off peak)
- price_peak_fix = price of power for the 2nd period (peak)
- price_mid_peak_fix = price of power for the 3rd period (mid peak)

---
## 3. Feature Engineering

### Annual Price Rate Differentials (January vs December)
> **New Features**: Changes in Time-of-Use Rates
> - Peak Rate Delta
> - Mid-Peak Rate Delta
> - Off-Peak Rate Delta
>
> **Hypothesis**: Significant annual price changes between January and December may impact customer churn decisions.

**Data Preprocessing: Converting Date Fields to Datetime Format**

In [7]:
client_df["date_activ"] = pd.to_datetime(client_df["date_activ"], format='%Y-%m-%d')
client_df["date_end"] = pd.to_datetime(client_df["date_end"], format='%Y-%m-%d')
client_df["date_modif_prod"] = pd.to_datetime(client_df["date_modif_prod"], format='%Y-%m-%d')
client_df["date_renewal"] = pd.to_datetime(client_df["date_renewal"], format='%Y-%m-%d')

In [8]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'price_data.csv'

In [ ]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean','price_peak_var' : 'mean','price_peak_fix' : 'mean','price_mid_peak_var' : 'mean','price_mid_peak_fix' : 'mean' }).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1_off_peak', 'price_off_peak_fix': 'dec_2_off_peak','price_peak_var' : 'dec_1_peak','price_peak_fix' : 'dec_2_peak','price_mid_peak_var' : 'dec_1_mid_peak', 'price_mid_peak_fix' : 'dec_2_mid_peak' }), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1_off_peak'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2_off_peak'] - diff['price_off_peak_fix']
diff['peak_diff_dec_january_energy'] = diff['dec_1_peak'] - diff['price_peak_var']
diff['peak_diff_dec_january_power'] = diff['dec_2_peak'] - diff['price_peak_fix']
diff['midpeak_diff_dec_january_energy'] = diff['dec_1_mid_peak'] - diff['price_mid_peak_var']
diff['midpeak_diff_dec_january_power'] = diff['dec_2_mid_peak'] - diff['price_mid_peak_fix']

diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power','peak_diff_dec_january_energy','peak_diff_dec_january_power','midpeak_diff_dec_january_energy','midpeak_diff_dec_january_power']]
diff.head()

In [ ]:
df = pd.merge(client_df, diff, on='id')
df.head()

---

### Period-to-Period Rate Differentials
> **New Features**: Average Price Differences Between Time-of-Use Periods
> 
> **Gas Rates**:
> - Peak vs Off-Peak Difference
> - Mid-Peak vs Peak Difference
> - Off-Peak vs Mid-Peak Difference
>
> **Electricity Rates**:
> - Peak vs Off-Peak Difference
> - Mid-Peak vs Peak Difference
> - Off-Peak vs Mid-Peak Difference
>
> **Hypothesis**: To analyze how price spreads between different time-of-use periods might influence customer churn behavior.

In [ ]:
# Aggregate average prices per period by company
mean_prices = price_df.groupby(['id']).agg({
    'price_off_peak_var': 'mean', 
    'price_peak_var': 'mean', 
    'price_mid_peak_var': 'mean',
    'price_off_peak_fix': 'mean',
    'price_peak_fix': 'mean',
    'price_mid_peak_fix': 'mean'    
}).reset_index()

In [ ]:
# Calculate the mean difference between consecutive periods
mean_prices['peak_off_peak_var_mean_diff'] = mean_prices['price_peak_var'] - mean_prices['price_off_peak_var']
mean_prices['mid_peak_peak_var_mean_diff'] = mean_prices['price_mid_peak_var'] - mean_prices['price_peak_var']
mean_prices['off_peak_mid_peak_var_mean_diff'] = mean_prices['price_off_peak_var'] - mean_prices['price_mid_peak_var']
mean_prices['peak_off_peak_fix_mean_diff'] = mean_prices['price_peak_fix'] - mean_prices['price_off_peak_fix']
mean_prices['mid_peak_peak_fix_mean_diff'] = mean_prices['price_mid_peak_fix'] - mean_prices['price_peak_fix']
mean_prices['off_peak_mid_peak_fix_mean_diff'] = mean_prices['price_off_peak_fix'] - mean_prices['price_mid_peak_fix']

In [ ]:
columns = [
    'id', 
    'peak_off_peak_var_mean_diff',
    'mid_peak_peak_var_mean_diff', 
    'off_peak_mid_peak_var_mean_diff',
    'peak_off_peak_fix_mean_diff', 
    'mid_peak_peak_fix_mean_diff', 
    'off_peak_mid_peak_fix_mean_diff'
]
df = pd.merge(df, mean_prices[columns], on='id')
df.head()

The dec-jan feature may reveal macro patterns that occur over an entire year, whereas inter-time-period features may reveal patterns on a micro scale between months.

---

### Maximum of Monthly Period-to-Period Rate Differentials
> **New Features**: Maximum Price Differences Between Time-of-Use Periods
> 
> **Gas Rates**:
> - Peak vs Off-Peak Difference
> - Mid-Peak vs Peak Difference
> - Off-Peak vs Mid-Peak Difference
>
> **Electricity Rates**:
> - Peak vs Off-Peak Difference
> - Mid-Peak vs Peak Difference
> - Off-Peak vs Mid-Peak Difference
>
> **Hypothesis**: To analyze how biggest price spreads between different time-of-use periods might influence customer churn behavior.

In [ ]:
# Aggregate average prices per period by company
mean_prices_by_month = price_df.groupby(['id', 'price_date']).agg({
    'price_off_peak_var': 'mean', 
    'price_peak_var': 'mean', 
    'price_mid_peak_var': 'mean',
    'price_off_peak_fix': 'mean',
    'price_peak_fix': 'mean',
    'price_mid_peak_fix': 'mean'    
}).reset_index()

In [ ]:
mean_prices_by_month

In [ ]:

mean_prices_by_month['peak_off_peak_var_mean_diff'] = mean_prices_by_month['price_peak_var'] - mean_prices_by_month['price_off_peak_var']
mean_prices_by_month['mid_peak_peak_var_mean_diff'] = mean_prices_by_month['price_mid_peak_var'] - mean_prices_by_month['price_peak_var']
mean_prices_by_month['off_peak_mid_peak_var_mean_diff'] = mean_prices_by_month['price_off_peak_var'] - mean_prices_by_month['price_mid_peak_var']
mean_prices_by_month['peak_off_peak_fix_mean_diff'] = mean_prices_by_month['price_peak_fix'] - mean_prices_by_month['price_off_peak_fix']
mean_prices_by_month['mid_peak_peak_fix_mean_diff'] = mean_prices_by_month['price_mid_peak_fix'] - mean_prices_by_month['price_peak_fix']
mean_prices_by_month['off_peak_mid_peak_fix_mean_diff'] = mean_prices_by_month['price_off_peak_fix'] - mean_prices_by_month['price_mid_peak_fix']

In [ ]:
# Calculate the maximum monthly difference across time periods
max_diff_across_periods_months = mean_prices_by_month.groupby(['id']).agg({
    'peak_off_peak_var_mean_diff': 'min',
    'mid_peak_peak_var_mean_diff': 'min',
    'off_peak_mid_peak_var_mean_diff': 'max',
    'peak_off_peak_fix_mean_diff': 'min',
    'mid_peak_peak_fix_mean_diff': 'min',
    'off_peak_mid_peak_fix_mean_diff': 'max'
}).reset_index().rename(
    columns={
        'peak_off_peak_var_mean_diff': 'peak_off_peak_var_max_monthly_diff',
        'mid_peak_peak_var_mean_diff': 'mid_peak_peak_var_max_monthly_diff',
        'off_peak_mid_peak_var_mean_diff': 'off_peak_mid_peak_var_max_monthly_diff',
        'peak_off_peak_fix_mean_diff': 'peak_off_peak_fix_max_monthly_diff',
        'mid_peak_peak_fix_mean_diff': 'mid_peak_peak_fix_max_monthly_diff',
        'off_peak_mid_peak_fix_mean_diff': 'off_peak_mid_peak_fix_max_monthly_diff'
    }
)

In [ ]:
columns = [
    'id',
    'peak_off_peak_var_max_monthly_diff',
    'mid_peak_peak_var_max_monthly_diff',
    'off_peak_mid_peak_var_max_monthly_diff',
    'peak_off_peak_fix_max_monthly_diff',
    'mid_peak_peak_fix_max_monthly_diff',
    'off_peak_mid_peak_fix_max_monthly_diff'
]

df = pd.merge(df, max_diff_across_periods_months[columns], on='id')
df.head()

I thought that calculating the maximum price change between months and time periods would be a valuable feature to include, as I wanted to consider the perspective of a PowerCo client. For utility customers, sudden price fluctuations between months can be especially frustrating, and a significant increase within a short time frame could prompt them to explore other providers for better rates. Since our goal is to predict churn in this case, I believe this feature could offer meaningful insights.

---
### Tenure

How long a company has been a client of PowerCo.

In [ ]:
df['tenure'] = ((df['date_end'] - df['date_activ']) / np.timedelta64(1, 'D') / 365.25).astype(int)

In [ ]:
df.groupby(['tenure']).agg({'churn': 'mean'}).sort_values(by='churn', ascending=False).plot(kind='bar',rot=360,figsize=(10,5))
plt.title('Customer Churn Rate by Tenure Length (in Year)', weight = 'bold', fontsize = 20)
plt.show()

We can see that companies who have only been a client for 4 or less years are much more likely to churn compared to companies that have been a client for longer. Interestingly, the difference between 4 and 5 months is about 4%, which represents a large jump in likelihood for a customer to churn compared to the other differences between ordered tenure values. Perhaps this reveals that getting a customer to over 4 months tenure is actually a large milestone with respect to keeping them as a long term customer. 

This is an interesting feature to keep for modelling because clearly how long you've been a client, has a influence on the chance of a client churning.

---
### Transform `datetime` features into numeric values representing the number of months (as integers)

- months_activ = Number of months active until reference date (Jan 2016)
- months_to_end = Number of months of the contract left until reference date (Jan 2016)
- months_modif_prod = Number of months since last modification until reference date (Jan 2016)
- months_renewal = Number of months since last renewal until reference date (Jan 2016)

Dates as a datetime object are not useful for a predictive model, so we needed to use the datetimes to create some other features that may hold some predictive power. 

Using intuition, you could assume that a client who has been an active client of PowerCo for a longer amount of time may have more loyalty to the brand and is more likely to stay. Whereas a newer client may be more volatile. Hence the addition of the `months_activ` feature.

As well as this, if we think from the perspective of a client with PowerCo, if you're coming toward the end of your contract with PowerCo your thoughts could go a few ways. You could be looking for better deals for when your contract ends, or you might want to see out your contract and sign another one. One the other hand if you've only just joined, you may have a period where you're allowed to leave if you're not satisfied. Furthermore, if you're in the middle of your contract, their may be charges if you wanted to leave, deterring clients from churning mid-way through their agreement. So, I think `months_to_end` will be an interesting feature because it may reveal patterns and behaviours about timing of churn.

My belief is that if a client has made recent updates to their contract, they are more likely to be satisfied or at least they have received a level of customer service to update or change their existing services. I believe this to be a positive sign, they are an engaged customer, and so I believe `months_modif_prod` will be an interesting feature to include because it shows the degree of how 'engaged' a client is with PowerCo.

Finally the number of months since a client last renewed a contract I believe will be an interesting feature because once again, it shows the degree to which that client is engaged. It also goes a step further than just engagement, it shows a level of commitment if a client renews their contract. For this reason, I believe `months_renewal` will be a good feature to include.

In [ ]:
def convert_months(reference_date, df, column):
    """
    Input a column with timedeltas and return months
    """
    time_delta = reference_date - df[column]
    months = (time_delta / np.timedelta64(1, 'D') / 30.44).astype(int)
    return months


In [ ]:
# Create reference date
reference_date = datetime(2016, 1, 1)

# Create columns
df['months_activ'] = convert_months(reference_date, df, 'date_activ')
df['months_to_end'] = -convert_months(reference_date, df, 'date_end')
df['months_modif_prod'] = convert_months(reference_date, df, 'date_modif_prod')
df['months_renewal'] = convert_months(reference_date, df, 'date_renewal')

In [ ]:
# We no longer need the datetime columns that we used for feature engineering, so we can drop them
remove = [
    'date_activ',
    'date_end',
    'date_modif_prod',
    'date_renewal'
]

df = df.drop(columns=remove)
df.head()

In [ ]:
columns = ['months_activ','months_to_end','months_modif_prod','months_renewal']
x_tick_rot_list = [45,360,45,360]
x_tick_font_size_list = [10,12,10,12]

for col,x_tick_rot,x_tick_font_size in zip(columns,x_tick_rot_list,x_tick_font_size_list) : 
    df.groupby([col]).agg({'churn': 'mean'}).sort_values(by='churn', ascending=False).plot(kind='bar',rot=x_tick_rot, figsize = (25,8))
    plt.title(f'Customer Churn Rate by {col} ', weight = 'bold', fontsize = 20)
    plt.xticks(fontsize=x_tick_font_size)
    plt.show()

there might be association between months_activ/months_modif_prod/months_renewal and churn

---
## 4. Data Preprocessing
### Encoding Categorical Data for Predictive Modeling

A predictive model cannot accept categorical or `string` values, hence as a data scientist you need to encode categorical features into numerical representations in the most compact and discriminative way possible.

The simplest method is to map each category to an integer (label encoding), however this is not always appropriate beecause it then introduces the concept of an order into a feature which may not inherently be present `0 < 1 < 2 < 3 ...`

Another way to encode categorical features is to use `dummy variables` AKA `one hot encoding`. This create a new feature for every unique value of a categorical column, and fills this column with either a 1 or a 0 to indicate that this company does or does not belong to this category.

#### has_gas

Transform this column from being categorical to being a binary flag

In [ ]:
df['has_gas'] = df['has_gas'].replace(['t', 'f'], [1, 0])
df.groupby(['has_gas']).agg({'churn': 'mean'}).plot(kind='bar',rot=360, figsize=(10,8))
plt.title('Customer Churn Rate by Gas Service Subscription Status', weight = 'bold', fontsize = 20)
plt.show()

If a customer also buys gas from PowerCo, it shows that they have multiple products and are a loyal customer to the brand. Hence, it is no surprise that customers who do not buy gas are almost 2% more likely to churn than customers who also buy gas from PowerCo. Hence, this is a useful feature.

#### channel_sales

In [ ]:
# Transform into categorical type
df['channel_sales'] = df['channel_sales'].astype('category')

# Let's see how many categories are within this column
df['channel_sales'].value_counts()

We have 8 categories, so we will create 8 dummy variables from this column. However, as you can see the last 3 categories in the output above, show that they only have 11, 3 and 2 occurrences respectively. Considering that our dataset has about 14000 rows, this means that these dummy variables will be almost entirely 0 and so will not add much predictive power to the model at all (since they're almost entirely a constant value and provide very little).

For this reason, we will drop these 3 dummy variables.

In [ ]:
df = pd.get_dummies(df, columns=['channel_sales'], prefix='channel')
df = df.drop(columns=['channel_sddiedcslfslkckwlfkdpoeeailfpeds', 'channel_epumfxlbckeskwekxbiuasklxalciiuu', 'channel_fixdbufsefwooaasfcxdxadsiekoceaa'])
df.head()

#### origin_up

In [ ]:
# Transform into categorical type
df['origin_up'] = df['origin_up'].astype('category')

# Let's see how many categories are within this column
df['origin_up'].value_counts()

Similar to `channel_sales` the last 3 categories in the output above show very low frequency, so we will remove these from the features after creating dummy variables.

In [ ]:
df = pd.get_dummies(df, columns=['origin_up'], prefix='origin_up')
df = df.drop(columns=['origin_up_MISSING', 'origin_up_usapbepcfoloekilkwsdiboslwaxobdp', 'origin_up_ewxeelcelemmiwuafmddpobolfuxioce'])
df.head()

---
### Addressing Skewness in Numerical Data

In the previous exercise we saw that some variables were highly skewed. The reason why we need to treat skewness is because some predictive models have inherent assumptions about the distribution of the features that are being supplied to it. Such models are called `parametric` models, and they typically assume that all variables are both independent and normally distributed. 

Skewness isn't always a bad thing, but as a rule of thumb it is always good practice to treat highly skewed variables because of the reason stated above, but also as it can improve the speed at which predictive models are able to converge to its best solution.

There are many ways that you can treat skewed variables. You can apply transformations such as:
- Square root
- Cubic root
- Logarithm

to a continuous numeric column and you will notice the distribution changes. For this use case we will use the 'Logarithm' transformation for the positively skewed features. 

<b>Note:</b> We cannot apply log to a value of 0, so we will add a constant of 1 to all the values

First I want to see the statistics of the skewed features, so that we can compare before and after transformation

#### original columns in dataset

In [ ]:
skewed = [
    'cons_12m', 
    'cons_gas_12m', 
    'cons_last_month',
    'forecast_cons_12m', 
    'forecast_cons_year', 
    'forecast_discount_energy',
    'forecast_meter_rent_12m', 
    'forecast_price_energy_off_peak',
    'forecast_price_energy_peak', 
    'forecast_price_pow_off_peak'
]

# Calculate skewness for each column in the skewed list
skewness_values = df[skewed].apply(lambda x: skew(x.dropna()))

# Display skewness values
skewness_values


We can see that the skewness for most of these features is quite high.

In [ ]:
# Apply log10 transformation
df["cons_12m"] = np.log10(df["cons_12m"] + 1)
df["cons_gas_12m"] = np.log10(df["cons_gas_12m"] + 1)
df["cons_last_month"] = np.log10(df["cons_last_month"] + 1)
df["forecast_cons_12m"] = np.log10(df["forecast_cons_12m"] + 1)
df["forecast_cons_year"] = np.log10(df["forecast_cons_year"] + 1)
df["forecast_meter_rent_12m"] = np.log10(df["forecast_meter_rent_12m"] + 1)
df["imp_cons"] = np.log10(df["imp_cons"] + 1)

In [ ]:
# Calculate skewness for each column in the skewed list
skewness_values = df[skewed].apply(lambda x: skew(x.dropna()))

# Display skewness values
skewness_values

Now we can see that for the majority of the features, their skewness is much lower after transformation. This is a good thing, it shows that these features are more stable and predictable now.

Let's quickly check the distributions of some of these features too.

In [ ]:
fig, axs = plt.subplots(nrows=3, figsize=(18, 20))


# Plot histograms
sns.distplot((df["cons_12m"].dropna()), ax=axs[0])
sns.distplot((df[df["has_gas"]==1]["cons_gas_12m"].dropna()), ax=axs[1])
sns.distplot((df["cons_last_month"].dropna()), ax=axs[2])

axs[0].set_title("Log-Transformed Distribution of cons_12m", fontsize=20, weight = 'bold', pad=15)
axs[1].set_title("Log-Transformed Distribution of cons_gas_12m", fontsize=20, weight = 'bold', pad=15)
axs[2].set_title("Log-Transformed Distribution of cons_last_month", fontsize=20, weight = 'bold', pad=15)
plt.subplots_adjust(hspace=0.4)
plt.show()

#### columns derived from feature engineering

In [ ]:
columns = [ 'offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power',
       'peak_diff_dec_january_energy', 'peak_diff_dec_january_power',
       'midpeak_diff_dec_january_energy', 'midpeak_diff_dec_january_power',
       'peak_off_peak_var_mean_diff', 'mid_peak_peak_var_mean_diff',
       'off_peak_mid_peak_var_mean_diff', 'peak_off_peak_fix_mean_diff',
       'mid_peak_peak_fix_mean_diff', 'off_peak_mid_peak_fix_mean_diff',
       'peak_off_peak_var_max_monthly_diff',
       'mid_peak_peak_var_max_monthly_diff',
       'off_peak_mid_peak_var_max_monthly_diff',
       'peak_off_peak_fix_max_monthly_diff',
       'mid_peak_peak_fix_max_monthly_diff',
       'off_peak_mid_peak_fix_max_monthly_diff']

# Calculate skewness for each column in the skewed list
skewness_values = df[columns].apply(lambda x: skew(x.dropna()))

# Display skewness values
skewness_values



Since the skewness values for most of these columns are not extreme, so only apply transformations to 'offpeak_diff_dec_january_power'

In [ ]:
df["offpeak_diff_dec_january_power"] = np.log10(df["offpeak_diff_dec_january_power"] + 1)

In [ ]:
skewness_values = df[['offpeak_diff_dec_january_power']].apply(lambda x: skew(x.dropna()))
skewness_values

In [ ]:
fig, axs = plt.subplots(nrows=1, figsize=(18, 10))
sns.distplot((df['offpeak_diff_dec_january_power'].dropna()), ax=axs)
axs.set_title("Log-Transformed Distribution of offpeak_diff_dec_january_power", fontsize=20, weight = 'bold', pad=15)

plt.show()

---
### Correlation Analysis

Another thing that is always useful to look at is how correlated all of the features are within your dataset.

One of the assumptions of any parametric predictive model (as stated earlier) is that all features must be independent.

For features to be independent, this means that each feature must have absolutely no dependence on any other feature. If two features are highly correlated and share similar information, this breaks this assumption. 

Ideally, you want a set of features that have 0 correlation with all of the independent variables (all features except our target variable) and a high correlation with the target variable (churn). However, this is very rarely the case and it is common to have a small degree of correlation between independent features.

So now let's look at how all the features within the model are correlated.

In [ ]:
columns = ['cons_12m', 'cons_gas_12m', 'cons_last_month',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'pow_max', 'churn',
       'offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power',
       'peak_diff_dec_january_energy', 'peak_diff_dec_january_power',
       'midpeak_diff_dec_january_energy', 'midpeak_diff_dec_january_power',
       'peak_off_peak_var_mean_diff', 'mid_peak_peak_var_mean_diff',
       'off_peak_mid_peak_var_mean_diff', 'peak_off_peak_fix_mean_diff',
       'mid_peak_peak_fix_mean_diff', 'off_peak_mid_peak_fix_mean_diff',
       'peak_off_peak_var_max_monthly_diff',
       'mid_peak_peak_var_max_monthly_diff',
       'off_peak_mid_peak_var_max_monthly_diff',
       'peak_off_peak_fix_max_monthly_diff',
       'mid_peak_peak_fix_max_monthly_diff',
       'off_peak_mid_peak_fix_max_monthly_diff', 'tenure', 'months_activ',
       'months_to_end', 'months_modif_prod', 'months_renewal']

correlation_matrix = df[columns].corr(numeric_only=True)

In [ ]:
fig = plt.figure(figsize=(70,70))
sns.heatmap(correlation_matrix,cmap="YlGnBu",annot_kws={'size': 27}, annot=True)
plt.title('Correlation Matrix of all variables', weight = 'bold', fontsize = 60)
plt.xticks(fontsize=30,rotation=45)
plt.yticks(fontsize=30)
plt.show()

Since there are too many columns in dataframe, the heatmap is a bit messy. Therefore, I'll filter out pairs of variables which have high correlation manually

In [ ]:
# Find pairs with correlation > 0.9 (excluding self-correlation of 1.0)
high_correlation_pairs = (
    correlation_matrix
    .where((abs(correlation_matrix) > 0.9) & (correlation_matrix != 1.0))
    .stack()
    .reset_index()
)

# Rename columns for better readability
high_correlation_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']

high_correlation_pairs['Sorted Pair'] = high_correlation_pairs.apply(
    lambda row: tuple(sorted([row['Variable 1'], row['Variable 2']])), axis=1
)

unique_high_correlation_pairs = high_correlation_pairs.drop_duplicates(subset=['Sorted Pair']).drop(columns=['Sorted Pair'])
# Display high-correlation pairs
unique_high_correlation_pairs = unique_high_correlation_pairs.sort_values(by=['Variable 1','Variable 2']).reset_index(drop=True)
unique_high_correlation_pairs

#### Suggestions for Dropping Columns
There are lots of variables pairs with high correlation. Therefore, we would want to drop some variables to ensure the independence between variables.

drop 
- has_gas
- forecast_cons_year
- forecast_price_energy_peak
- margin_gross_pow_ele
- mid_peak_peak_fix_mean_diff
- midpeak_diff_dec_january_energy
- num_years_antig
- months_activ
- off_peak_mid_peak_fix_mean_diff
- off_peak_mid_peak_var_max_monthly_diff
- off_peak_mid_peak_var_mean_diff
- peak_diff_dec_january_power
- peak_off_peak_fix_max_monthly_diff
- peak_off_peak_fix_mean_diff

In [ ]:
columns_to_drop = [
    'has_gas',
    'forecast_cons_year',
    'forecast_price_energy_peak',
    'margin_gross_pow_ele',
    'mid_peak_peak_fix_mean_diff',
    'midpeak_diff_dec_january_energy',
    'num_years_antig',
    'months_activ',
    'off_peak_mid_peak_fix_mean_diff',
    'off_peak_mid_peak_var_max_monthly_diff',
    'off_peak_mid_peak_var_mean_diff',
    'peak_diff_dec_january_power',
    'peak_off_peak_fix_max_monthly_diff',
    'peak_off_peak_fix_mean_diff'
]

df = df.drop(columns_to_drop, axis = 1)
df.head()

---
## 5. Export the data for modeling

In [ ]:
#df.to_csv('data_for_modeling.csv')